fd5d23725f8b05cc91cbca3848ac7437

# 작업내용

- 영화진흥위원회 OpenAPI를 활용하면 현재 극장에서 상영중인 영화들에 대한 박스오피스 데이터를 조회할 수 있습니다.
- 영화진흥원 OpenAPI를 통해 JSON 형식의 데이터를 수집하고 이를 그래프로 시각화 하는 내용을 소개합니다.

# Key 발급 받기

1. 영화진흥위원회 OpenAPI 사이트에 회원가입 후 로그인을 수행한다.
 - http://www.kobis.or.kr/kobisopenapi/homepg/main/main.do
 
 
2. 키 발급/관리 메뉴를 통해 연동에 필요한 인증키 발급받는다.

# 연동 스펙 확인하기
http://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do 페이지를 통해 요청에 필요한 정보와 응답 형식을 확인한다.

In [3]:
# 패키지 불러오기

library(httr)    # 온라인상의 데이터를 가져오기 위한 통신 기능 패키지
library(rjson)   # JSON 처리 패키지
library(dplyr)   # 데이터 정제 패키지
library(ggplot2) # 데이터 정제 패키지

NameError: name 'library' is not defined

# 데이터 수집하기

## API 접근에 필요한 URL 구성하기

### 요청변수로 사용할 하루 전 날짜값 만들기

In [ ]:
# 오늘 날짜 가져오기

today <- Sys.Date()
today

In [ ]:
# 날짜 계산

yesterday = today-1
yesterday

In [ ]:
# 시작 날짜 
startDt <- as.Date('2000/1/1',format =  "%Y/%m/%d")
# 마지막 날짜
endDt <- as.Date('2020/1/1',format =  "%Y/%m/%d")

In [ ]:
while(startDt != endDt){
    
    print(startDt)
    startDt <- startDt+1
}

### 날짜 형식 지정
%d : 숫자형식의 날짜   
%m : 숫자형식의 월   
%b : 약자 형식의 월 이름   
%B : 월에 대한 영문 이름   
%y : 2자리 숫자 형식   
%Y : 4자리 숫자 형식   

In [ ]:
targetDt <- format(yesterday, "%Y%m%d")
targetDt

## API키와 날짜를 조합하여 접속할 URL 구성

In [ ]:
# 영화 진흥원에서 발급받은 API키
kobis_api_key = "fd5d23725f8b05cc91cbca3848ac7437"

# 영화진흥원 API URL
#kobis_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=N&repNationCd=K"
kobis_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=N&repNationCd=K"

# `%s`로 지정된 부분에 변수값을 치환하여 전체 주소 결정
api_url <- sprintf(kobis_api_url, kobis_api_key, targetDt)
api_url


## OpenAPI와 연동하여 데이터 수집하기

In [ ]:
# API 접속 후 데이터 수집하기

resp <- GET(api_url)
resp


In [ ]:
# 수집 결과를 데이터프레임으로 변환

# 리스트로 변환하기
resp_dat <- content(resp, as="parse", encoding="utf-8")
# 리스트에서 배열 부분만 추출하여 데이터프레임으로 변환
resp_df <- bind_rows(resp_dat$boxOfficeResult$dailyBoxOfficeList)
resp_df[1,]

# MovieCd,MovieNm,openDt,audiCnt(해당일 관람객수),audiAcc(누적관람객수),scrnCnt(해당일 스크린수),showCnt(해당일 상영 횟수)

In [ ]:
df<- data.frame(Date = as.Date(substr(resp_dat$boxOfficeResult$showRange,1,8),"%Y%m%d"), resp_df[,c(5,6,7,13,16,17,18)])

In [ ]:
# 시작 날짜 
startDt <- as.Date('2004/1/1',format =  "%Y/%m/%d")
# 마지막 날짜
endDt <- as.Date('2020/1/1',format =  "%Y/%m/%d")
# 타겟 날짜
targetDt <- format(startDt, "%Y%m%d")


In [ ]:
# 시작날짜와 종료 날짜까지 api 받아오기
df <- c()
while(startDt != endDt){
    targetDt <- format(startDt, "%Y%m%d")
    
    api_url <- sprintf(kobis_api_url, kobis_api_key, targetDt)
    resp <- GET(api_url)
    
    # 리스트로 변환하기
    resp_dat <- content(resp, as="parse", encoding="utf-8")
    # 리스트에서 배열 부분만 추출하여 데이터프레임으로 변환
    resp_df <- bind_rows(resp_dat$boxOfficeResult$dailyBoxOfficeList)
    temp_df<- data.frame(Date = as.Date(substr(resp_dat$boxOfficeResult$showRange,1,8),"%Y%m%d"), resp_df[,c(5,6,7,13,16,17,18)])
    df <- rbind(df,temp_df)
    
    startDt <- startDt+1
    
}


In [ ]:
head(df)

In [ ]:
tail(df)

In [ ]:
    # 시작 날짜 
    startDt <- as.Date('2004/1/1',format =  "%Y/%m/%d")
    # 타겟 날짜
    targetDt <- format(startDt, "%Y%m%d")


    api_url <- sprintf(kobis_api_url, kobis_api_key,targetDt )
    resp <- GET(api_url)
    
    # 리스트로 변환하기
    resp_dat <- content(resp, as="parse", encoding="utf-8")
    # 리스트에서 배열 부분만 추출하여 데이터프레임으로 변환
    resp_df <- bind_rows(resp_dat$boxOfficeResult$dailyBoxOfficeList)
    temp_df<- data.frame(Date = as.Date(substr(resp_dat$boxOfficeResult$showRange,1,8),"%Y%m%d"), resp_df[,c(5,6,7,13,16,17,18)])


In [ ]:
temp_df

# 데이터 전처리
## 필요한 데이터만 추출

영화제목과 관람객 수 데이터만 추출후 결과 확인.   
관람객 수 데이터가 문자열(chr)임이 확인된다. 이는 숫자가 아니므로 통계값 산출이 불가능하기 때문에 변환이 필요하다.

In [ ]:
boxoffice_df <- resp_df %>% select(movieNm, audiCnt)
boxoffice_df

In [ ]:
# 관람객 수 데이터 타입을 integer로 변경

boxoffice_df$audiCnt <- as.integer(boxoffice_df$audiCnt)
boxoffice_df

In [ ]:
# 컬럼이름 변경

movie_audience_df <- rename(boxoffice_df, '영화제목'=movieNm, '관람객수'=audiCnt)
movie_audience_df

# 데이터 시각화

In [ ]:
# 시각화 환경 설정

options(repr.plot.width=20, repr.plot.height=10, warn=-1)

In [ ]:
ggplot(data=movie_audience_df) +
    geom_col(aes(x=영화제목, y=관람객수, fill=영화제목)) +
    # 가로 막대 그래프로 설정
    coord_flip() +
    # 배경을 흰색으로 설정
    theme_bw() +
    # 그래프 타이틀 설정
    ggtitle(sprintf("%s 영화별 관람객 수", yesterday)) +
    # x축 제목 설정
    xlab("영화제목") +
    # y축 제목 설정
    ylab("관람객수") +
    # y축 값의 간격 및 세자리마다 콤마 적용
    scale_y_continuous(breaks=seq(0, max(movie_audience_df$관람객수), 20000), labels=scales::comma) +
    # 각 텍스트의 색상, 크기, 각도, 글꼴 설정
    theme(plot.title=element_text(family="NanumGothic", color="#0066ff", size=25, face="bold", hjust=0.5),
          axis.title.x=element_text(family="NanumGothic", color="#999999", size=18, face="bold"),
          axis.title.y=element_text(family="NanumGothic", color="#999999", size=18, face="bold"),
          axis.text.x=element_text(family="NanumGothic", color="#000000", size=16, angle=0),
          axis.text.y=element_text(family="NanumGothic", color="#000000", size=16, angle=0)) +
    # 범례 지우기
    theme(legend.position = "none")

In [ ]:
read.csv("")